In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn import preprocessing
from scipy.sparse import csr_matrix
import pickle
import os
from annoy import AnnoyIndex

In [2]:
patient_icd_sparse_path = "./../../data/PATIENT_ICD_BINARY_SPARSE_CSR.p"
pdata = pickle.load(open(patient_icd_sparse_path, "rb"))
pdata.shape

(46520, 6984)

In [7]:
x = np.array([[4, 0, 0],[0,4,0],[0,0,4]])
np.sqrt(np.linalg.inv(x))

array([[0.5, 0. , 0. ],
       [0. , 0.5, 0. ],
       [0. , 0. , 0.5]])

In [5]:
ANNOY_PATH = "../../../large_data_files"
annoy_path = os.path.join(ANNOY_PATH, "200214_patient_similarity_clusters_default.ann")

cos_knn_tree = AnnoyIndex(pdata.shape[1], "angular")
cos_knn_tree.load(annoy_path)
k_neighbors = 10
k_neighbors += 1

print("Building dense data matrix with k={} nn...".format(k_neighbors))
A = np.zeros((pdata.shape[0], pdata.shape[0]))
D = np.zeros((pdata.shape[0], pdata.shape[0]))

print(knn_data.shape)
for i in tqdm(range(pdata.shape[0])):
    nn_idxs = cos_knn_tree.get_nns_by_item(i, k_neighbors)[1:]
    distances = [cos_knn_tree.get_distance(i, nn) for nn in nn_idxs]
    print(i, nn_idxs)
    print(i, distances)
    D[i, i] = np.sum(distances)
    A[i, nn_idxs] = distances

D_inv_sqrt = np.sqrt(np.linalg.inv(D))
L = D_inv_sqrt @ A @ D_inv_sqrt
L_sparse = csr_matrix(L)
L_sparse.shape

  0%|          | 0/46520 [00:00<?, ?it/s]

Building dense data matrix with k=11 nn...
(46520, 46520)
0 [3319, 29662, 24132, 25275, 16666, 37755, 39245, 16376, 14844, 26326]
0 [1.1148737668991089, 1.1428152322769165, 1.1491882801055908, 1.163966417312622, 1.1917420625686646, 1.1946163177490234, 1.210144281387329, 1.212558627128601, 1.213560700416565, 1.213560700416565]


(46520, 46520)

In [ ]:
pickle.dump(L_sparse, open("PATIENT_SIM_COS_K{}_CSR.csv".format(k_neighbors - 1), "wb"))

In [ ]:
gt1_subject_ids_path = "../../data/PATIENT_ICD_SUBJECT_IDS_GT1_VISITS.csv"
gt1_subject_ids = pd.read_csv(gt1_subject_ids_path)
gt1_idxs = gt1_subject_ids.MATRIX_ROW_IDX.values

patient_icd_sparse_path = "./../../data/PATIENT_ICD_BINARY_SPARSE_CSR.p"
pdata_sparse = pickle.load(open(patient_icd_sparse_path, "rb"))
pdata_dense = pdata_sparse.toarray()

pdata_gt1 = pdata_dense[gt1_idxs]
pdata_gt1.shape

In [ ]:
##Build affinity matrix


In [ ]:
unnormalized_data = pdata_dense
print(unnormalized_data.shape)
std_data = preprocessing.scale(unnormalized_data)
pca = PCA().fit(std_data)

In [ ]:
print(np.sum(pca.explained_variance_ratio_))
print(pca.explained_variance_ratio_)

In [ ]:
%%timeit -n 1 -r 1
unnormalized_data = pdata_gt1
std_data = preprocessing.scale(unnormalized_data)

n_clusters = 5
sc = SpectralClustering(n_clusters=n_clusters).fit(std_data)
unique, counts = np.unique(sc.labels_, return_counts=True)
print(unique, counts)